In [1]:
import numpy as np
import pickle
from collections import Counter
import re

# nlp libraries/api
import en_core_web_lg
from spacy import displacy
import gensim
#from neuralcoref import Coref

spacy = en_core_web_lg.load()
#coref = Coref(nlp=spacy)


In [3]:
# Load opinion lexicon
neg_file = open("neg_words.txt",encoding = "ISO-8859-1")
pos_file = open("pos_words.txt",encoding = "ISO-8859-1")
neg = [line.strip() for line in neg_file.readlines()]
pos = [line.strip() for line in pos_file.readlines()]
opinion_words = neg + pos


In [11]:

def check_similarity(aspects, word):
    '''
    checks for word2vec similarity values between category word and the term
    returns most similar word
    '''
    similarity = []
    for aspect in aspects:
        similarity.append(word2vec.n_similarity([aspect], word.split()))
    # set threshold for max value
    if max(similarity) > 0.2:
        return aspects[np.argmax(similarity)]
    else:
        return None


def assign_term_to_aspect(aspect_sent, terms_dict, sent_dict, pred):
    '''
    function: takes in a sentiment dictionary and appends the aspect dictionary
    inputs: sent_dict is a Counter in the form Counter(term:sentiment value)
            aspect_sent is total sentiment tally
            terms_dict is dict with individual aspect words associated with sentiment
    output: return two types of aspect dictionaries:
            updated terms_dict and aspect_sent
    '''
    aspects = ['location', 'checkin', 'food', 'building', 'room', 'cleanliness', 'value', 'service', 'business']

    # First, check word2vec
    # Note: the .split() is used for the term because word2vec can't pass compound nouns
    for term in sent_dict:
        try:
            # The conditions for when to use the NB classifier as default vs word2vec
            aspect = check_similarity(aspects, term)
            if aspect:
                terms_dict[aspect.upper()][term] += sent_dict[term]
                if sent_dict[term] > 0:
                    aspect_sent[aspect.upper()]["pos"] += sent_dict[term]
                else:
                    aspect_sent[aspect.upper()]["neg"] += abs(sent_dict[term])
            elif pred:
                aspect = str(pred[0]).strip("(',)")
                terms_dict[aspect.upper()][term] += sent_dict[term]
                if sent_dict[term] > 0:
                    aspect_sent[aspect.upper()]["pos"] += sent_dict[term]
                else:
                    aspect_sent[aspect.upper()]["neg"] += abs(sent_dict[term])
            # if unable to classify via NB or word2vec, then put them in misc. bucket
            else:
                terms_dict["OTHER"][term] += sent_dict[term]
                if sent_dict[term] > 0:
                    aspect_sent["OTHER"]["pos"] += sent_dict[term]
                else:
                    aspect_sent["OTHER"]["neg"] += abs(sent_dict[term])
        except:
            print(term, "not in vocab")
            continue
    return aspect_sent, terms_dict


def modify_sentiment(token, sentiment):
    for child in token.children:
        # if there's a adj modifier (i.e. very, pretty, etc.) add more weight to sentiment
        # This could be better updated for modifiers that either positively or negatively emphasize
        # can't catch "there are nowhere near enough loungers"
        if ((child.dep_ == "amod") or (child.dep_ == "advmod")) and (child.text in opinion_words):
            sentiment *= 1.5
        # check for negation words and flip the sign of sentiment
        if child.dep_ == "neg":
            sentiment *= -1
    return sentiment


def add_subchild(child, sent_dict, sentiment, token, added):
    if child.children == []:
        return sent_dict, added
    else:
        conj = 0
        for subchild in child.children:
            if (subchild.dep_ in ["compound", "amod", "nmod"]) & (subchild != token):
                sent_dict[subchild.lemma_ + ' ' + child.lemma_] += sentiment
                for sub in subchild.children:
                    if sub.pos_ in ["NOUN", "PROPN"]:
                        sent_dict[sub.lemma_ + ' ' + child.lemma_] += sentiment
                added = True
            # check for conjugates (a AND b), then add both to dictionary
            elif subchild.text == "and" or subchild.text == ",":
                conj += 1
        for subchild in child.children:
            if (conj > 0) & (subchild.pos_ == "NOUN" or subchild.pos_ == "PROPN"):
                sent_dict = check_compound(subchild, sent_dict, sentiment, child)
                conj -= 1
    return sent_dict, added


def check_compound(child, sent_dict, sentiment, token=None):
    added = False
    sent_dict, added = add_subchild(child, sent_dict, sentiment, token, added)
    if not added:
        sent_dict[child.lemma_] += sentiment
    return sent_dict


def for_compound(token, sent_dict, sentiment):
    if token.head.dep_ == "compound":
        sent_dict = for_compound(token.head, sent_dict, sentiment)
    elif token.head.pos_ in ["NOUN", "PROPN"]:
        sent_dict = check_compound(token.head, sent_dict, sentiment)
    return sent_dict


def for_acomp(token, sent_dict, sentiment):
    for ancestor in token.ancestors:
        if ancestor.dep_ == "attr":
            sent_dict[ancestor] += sentiment
        if ancestor.dep_ in ["advcl", "ROOT", "conj"]:
            sent_dict = for_advcl(ancestor, sent_dict, sentiment, token)
    return sent_dict


def for_advcl(ancestor, sent_dict, sentiment, token):
    for child in ancestor.children:
        if (child.dep_ in ["nsubj", "compound"] ) & (child.pos_ in ["NOUN", "PROPN"]):
            sent_dict = check_compound(child, sent_dict, sentiment, token)
    return sent_dict


def for_amod(token, sent_dict, sentiment):
    if token.head.dep_ == "appos":
        sent_dict = check_compound(token.head, sent_dict, sentiment, token)
    elif token.head.dep_ == "pobj":
        if token.head.head.head.dep_ in ["advcl", "ROOT", "conj"]:
            sent_dict = for_advcl(token.head.head.head, sent_dict, sentiment, token)
    if token.head.pos_ == "NOUN" or token.head.pos_ == "PROPN":  # token.head.dep_ != "compound", so "free room delivery" not added twice
        sent_dict = check_compound(token.head, sent_dict, sentiment, token)
    return sent_dict


def for_dobj(token, sent_dict, sentiment):
    if token.head.dep_ == "conj":
        head = token.head.head
        head_child = token.head
    else:
        head = token.head
        head_child = token
    for child in head.children:
        if (child.dep_ == "nsubj") & (child.pos_ == "NOUN" or child.pos_ == "PROPN"):
            sent_dict = check_compound(child, sent_dict, sentiment, head_child)
    return sent_dict


def for_prep(token, sent_dict, sentiment):
    for child in token.children:
        if child.pos_ == "NOUN" or child.pos_ == "PROPN":
            sent_dict = check_compound(child, sent_dict, sentiment, token)
    return sent_dict


def feature_sentiment(sentence):
    '''
    input: dictionary and sentence
    function: appends dictionary with new features if the feature did not exist previously,
              then updates sentiment to each of the new or existing features
    output: updated dictionary
    '''
    sent_dict = Counter()
    sentence = spacy(sentence)
    for token in sentence:
        # check if the word is an opinion word, then assign sentiment
        if (token.lemma_ in opinion_words) or (token.text in opinion_words):
            sentiment = 1 if ((token.text in pos) or (token.lemma_ in pos)) else -1
            # if target is an adverb modifier (i.e. pretty, highly, etc.)
            # but happens to be an opinion word, ignore and pass
            sentiment = modify_sentiment(token, sentiment)
            if (token.dep_ == "advmod"):
                continue
            elif (token.dep_ == "acomp"):
                sent_dict = for_acomp(token, sent_dict, sentiment)
            elif (token.dep_ == "amod"):
                sent_dict = for_amod(token, sent_dict, sentiment)
            elif (token.dep_ == "compound"):
                sent_dict = for_compound(token, sent_dict, sentiment)
            elif (token.dep_ == "dobj"):
                sent_dict = for_dobj(token, sent_dict, sentiment)
            elif (token.dep_ == "relcl"):
                if token.head.pos_ == "NOUN" or token.head.pos_ == "PROPN":
                    sent_dict[token.head.lemma_] += sentiment
            else:
                    # if verb, check if there's a direct object
                if token.head.dep_ == "advcl" or token.head.dep_ == "ROOT":
                    sent_dict = for_advcl(token.head, sent_dict, sentiment, token)
                for child in token.children:
                    if (token.pos_ == "VERB") & (child.dep_ == "dobj"):
                        sent_dict = check_compound(child, sent_dict, sentiment, token)
                    # check for nouns
                    elif child.pos_ == "NOUN" or child.pos_ == "PROPN":
                        sent_dict = check_compound(child, sent_dict, sentiment, token)
                    if ((token.pos_ == "VERB") or (token.pos_ == "ADJ")) & (child.dep_ == "prep"):
                        sent_dict = for_prep(child, sent_dict, sentiment)
    return sent_dict


def classify_and_sent(sentence, aspect_sent, terms_dict):
    '''
    function: classify the sentence into a category, and assign sentiment
    note: aspect_dict is a parent dictionary with all the aspects
    input: sentence & aspect dictionary, which is going to be updated
    output: updated aspect dictionary
    '''
    # classify sentence with NB classifier
    predicted = svm_model.predict([sentence])
    pred = mlb.inverse_transform(predicted)
    if "('OTHER')," in pred:
        pred.remove("('OTHER'),")
    if "('NOTRELATED')," in pred:
        pred.remove("('NOTRELATED'),")

    # get aspect names and their sentiment in a dictionary form
    sent_dict = feature_sentiment(sentence)

    # try to categorize the aspect names into the 4 aspects in aspect_dict
    aspect_sent, terms_dict = assign_term_to_aspect(aspect_sent, terms_dict, sent_dict, pred)
    return aspect_sent, terms_dict


#def replace_pronouns(text):
 #   coref.one_shot_coref(text)
  #  return coref.get_resolved_utterances()[0]

from nltk.tokenize import sent_tokenize

def split_sentence(text):
    '''
    splits review into a list of sentences using spacy's sentence parser
    '''
    review = spacy(text)
    bag_sentence = []
    start = 0
    for token in review:
        if token.sent_start:
            bag_sentence.append(review[start:(token.i - 1)])
            start = token.i
        if token.i == len(review) - 1:
            bag_sentence.append(review[start:(token.i + 1)])
    return bag_sentence

# Remove special characters using regex
def remove_special_char(sentence):
    return re.sub(r"[^a-zA-Z0-9.',:;?]+", ' ', sentence)
# "ROOMS", "CLEANLINESS", "VALUE", "SERVICE", "LOCATION", "CHECKIN", "BUSINESS", "FOOD", "BUILDING", "OTHER", "NOTRELATED"
def review_pipe(review, aspect_sent, terms_dict):
    #review = replace_pronouns(review)
    sentences = split_sentence(review)
    for sentence in sentences:
        sentence = remove_special_char(str(sentence))
        aspect_sent, terms_dict = classify_and_sent(sentence.lower(), aspect_sent, terms_dict)
    return aspect_sent, terms_dict

In [5]:
# Uncomment below if running for first time.
# Setup nltk corpora path and Google Word2Vec location
#google_vec_file = 'GoogleNews-vectors-negative300.bin'
#word2vec = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)
#pickle.dump(word2vec, open("word2vec_google.pkl", 'wb'))

# If above script has been run, load saved word embedding
word2vec = pickle.load(open("word2vec_google.pkl", 'rb'))

# load the Multi-label binarizer from previous notebook
mlb = pickle.load(open("mlb.pkl", 'rb'))

# load the fitted naive bayes model from previous notebook
svm_model = pickle.load(open("svm_model.pkl", 'rb'))

In [6]:
with open("hotel_reviewVA.txt") as fp: 
    reviews = []
    Lines = fp.readlines() 
    for line in Lines: 
        reviews.append(line)

In [8]:
reviews[0]

'This hotel was nice and quiet. Did not know, there was train track near by. But it was only few train passed during our stay. Best Western changed hotel classification. The Plus category are not the same as before.\n'

In [12]:
aspect_sent={'LOCATION':Counter(), 'CHECKIN':Counter(), 'FOOD':Counter(), 'BUILDING':Counter(), 'ROOM':Counter(), 'CLEANLINESS':Counter(), 'VALUE':Counter(), 'SERVICE':Counter(), 'BUSINESS':Counter(), 'OTHER':Counter()}
term_dict={'LOCATION':Counter(), 'CHECKIN':Counter(), 'FOOD':Counter(), 'BUILDING':Counter(), 'ROOM':Counter(), 'CLEANLINESS':Counter(), 'VALUE':Counter(), 'SERVICE':Counter(), 'BUSINESS':Counter(), 'OTHER':Counter()}
for review in reviews:
    aspect_sent, term_dict = review_pipe(review, aspect_sent, term_dict)

attention not in vocab
rimmed glass not in vocab
-PRON- not in vocab
-PRON- not in vocab
order not in vocab
-PRON- not in vocab
-PRON- not in vocab
. location not in vocab
n.o location not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
callaisgeneral manager not in vocab
air con not in vocab
-PRON- not in vocab
russells not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
callaisgeneral manager not in vocab
advice not in vocab
-PRON- not in vocab
-PRON- not in vocab
callaisgeneral manager not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
sheet not in vocab
addition not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not in vocab
-PRON- not

In [14]:
aspect_sent

{'LOCATION': Counter({'pos': 93.5, 'neg': 10}),
 'CHECKIN': Counter({'pos': 71.5, 'neg': 5}),
 'FOOD': Counter({'neg': 14, 'pos': 108.0}),
 'BUILDING': Counter({'neg': 9, 'pos': 21.5}),
 'ROOM': Counter({'pos': 258.5, 'neg': 71.5}),
 'CLEANLINESS': Counter({'pos': 21.5, 'neg': 4}),
 'VALUE': Counter({'neg': 7, 'pos': 25.5}),
 'SERVICE': Counter({'neg': 25, 'pos': 129.5}),
 'BUSINESS': Counter({'pos': 18, 'neg': 2}),
 'OTHER': Counter({'pos': 121.0, 'neg': 21})}

In [13]:
term_dict

{'LOCATION': Counter({'western classification': 1,
          'excellent location': 1,
          'cheap location': 1,
          'difficult access': -2,
          'place': 3,
          'many place': 0,
          'other place': 0,
          'expensive place': 1,
          'attractive place': -1,
          'downtown location': 1,
          'good place': 1,
          'location': 21,
          'visit': 13,
          'close location': 4,
          'walk': 2,
          'person': 1,
          'facility': -1,
          'information': 1,
          'distance': 1,
          'status': 1,
          'perfect stay': 1,
          'trip': 1,
          'crescent city': 1,
          'walk distance': 1,
          'central district': 1,
          'saint game': 1,
          'heart': 1,
          'saint avenue': 1,
          'walking distance': 1,
          'city view': 1,
          'weekend getaway': 1,
          'town atmosphere': 0,
          'living area': 2,
          'common area': 1,
          'feature'